In [1]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [60]:
import psycopg2
from decouple import config

from queries_helper import query_states, query_corona_cases, query_insured_population, query_population
from herlper import load_tables, insert_states, load_corona_cases, load_insurance, load_population

SyntaxError: invalid syntax (<ipython-input-60-11fa3117663a>, line 5)

# Import Data 

## States

In [2]:
r = requests.get('https://www.faa.gov/air_traffic/publications/atpubs/cnt_html/appendix_a.html')
s = BeautifulSoup(r.text, 'lxml')

In [3]:
table = s.find('table')
table_of_states = [td.text.strip() for tr in table.find_all('tr') for td in tr.find_all('td') ] 

In [4]:
state_names = table_of_states[::2]
state_abbreviation  = table_of_states[1::2]

In [5]:
dict_state_names = {i[0].capitalize():i[1] for i in list(zip(state_names, state_abbreviation))}
dict_state_names['United states'] = 'All'
dict_state_names['US Total'] = 'All'

In [6]:
def change_name_abb(x:str, names:dict) -> str:
    try:
        abb = names[x.capitalize()]
    except KeyError:
        abb = None
    return abb
    

In [7]:
df_state_name = pd.DataFrame([dict_state_names]).T.reset_index()

In [8]:
df_state_name.columns = ['Name', 'Abbreviation ']

In [9]:
df_state_name.head()

,Name,Abbreviation
0,Alabama,AL
1,Kentucky,KY
2,Ohio,OH
3,Alaska,AK
4,Louisiana,LA


## Seguros 

In [10]:
path_data_uninsured_state = '//mnt/c/Users/cesar/Downloads/aspe-uninsured-estimates-by-state.xlsx'

In [11]:
xls    = pd.ExcelFile(path_data_uninsured_state)
df_us = pd.read_excel(xls, 'All Uninsured (#)')

In [12]:
df_us = df_us[['State Name', 'Total Non-Elderly Population (Excluding Undocumented)','Uninsured Population (Excluding Undocumented)', 'Male','Female', 'Age 0-18', 'Age 19-34', 'Age 35-49',
       'Age 50-64']]

In [13]:
df_us = df_us.rename(columns={
    'Total Non-Elderly Population (Excluding Undocumented)':'Non-Elderly Population',
    'Uninsured Population (Excluding Undocumented)':'Uninsured Population (Excluding Undocumented)'
})

In [14]:
df_us['State Name'] = df_us['State Name'].apply(lambda x : change_name_abb(x, dict_state_names))

In [15]:
df_us.head()

,State Name,Non-Elderly Population,Uninsured Population (Excluding Undocumented),Male,Female,Age 0-18,Age 19-34,Age 35-49,Age 50-64
0,AL,3986500,456100,253100,202900,36800,188500,128300,102400
1,AK,629300,80300,49500,30800,13300,37700,16200,13200
2,AZ,5720700,698800,397500,301300,144800,253800,166800,133300
3,AR,2443900,257200,149600,107600,37600,99200,72300,48100
4,CA,32060700,2397600,1373600,1024000,325600,968800,623300,479900


In [16]:
df_us.dropna(subset=['State Name'], inplace=True)

## State

In [17]:
path_state = '//mnt/c/Users/cesar/Downloads/acs2021_1yr_B27003_04000US09.xlsx'
df_state = pd.read_excel(path_state)
df_state = df_state[['Unnamed: 0', 'Alabama', 'Alaska',
       'Arizona', 'Arkansas', 'California',
       'Colorado',  'Connecticut',
        'Delaware',  'District of Columbia',
        'Florida',  'Georgia', 
       'Hawaii', 'Idaho', 'Illinois',
        'Indiana',  'Iowa', 
       'Kansas', 'Kentucky', 'Louisiana',
        'Maine',  'Maryland', 
       'Massachusetts',  'Michigan',
       'Minnesota', 'Mississippi', 
       'Missouri',  'Montana',  'Nebraska',
        'Nevada',  'New Hampshire',
        'New Jersey', 'New Mexico',
       'New York', 'North Carolina',
        'North Dakota',  'Ohio',
        'Oklahoma',  'Oregon', 
       'Pennsylvania',  'Rhode Island', 
       'South Carolina', 'South Dakota', 
       'Tennessee', 'Texas',  'Utah',
        'Vermont',  'Virginia',
       'Washington', 'West Virginia', 
       'Wisconsin',  'Wyoming' ]]

In [18]:
df_state.name = 'index'

In [19]:
df_state = df_state.set_index('Unnamed: 0').T.reset_index().rename(columns={'index':'State'})

In [20]:
df_state['State abbreviation'] = df_state['State'].apply(lambda x : change_name_abb(x, dict_state_names))

In [21]:
df_state.head()

Unnamed: 0,State,Total,Male,Female,State abbreviation
0,Alabama,4957633,2384150,2573483,AL
1,Alaska,702154,357463,344691,AK
2,Arizona,7174053,3546481,3627572,AZ
3,Arkansas,2974701,1457278,1517423,AR
4,California,38724296,19209230,19515064,CA


## COVID Cases

In [22]:
path_data_covid = '//mnt/c/Users/cesar/Downloads/covid_confirmed_usafacts.csv'

In [23]:
df_cd = pd.read_csv(path_data_covid)

In [24]:
df_cd.drop(columns=['County Name', 'countyFIPS'], inplace=True)

In [25]:
df_cd = df_cd.groupby(['State','StateFIPS']).sum().reset_index()

In [26]:
df_cd = df_cd.set_index(['State','StateFIPS']).sum(axis=1, numeric_only=True).reset_index().rename(columns={0:'Total cases'})

In [27]:
df_cd.head()

,State,StateFIPS,Total cases
0,AK,2,114436633
1,AL,1,706369074
2,AR,5,438592973
3,AZ,4,1081616913
4,CA,6,4660986565


# Load

In [29]:
dbname = 'usa_insurance'
user = config('db_user')
port = config('db_port')
host = config('db_host')
password = config('db_passoword')

In [57]:
connection = psycopg2.connect(
                    dbname = dbname,
                    host = host,
                    port = port,
                    user = user,
            password = password)

cursor = connection.cursor()

In [34]:
with connection.cursor() as curs:
    load_tables(curs)
    connection.commit()

## Insert States

In [42]:
load_population(cursor, connection, df_state_name)

## Insert Population

In [173]:
load_population(cursor, connection, df_state)

## Insert seguros

In [29]:
load_insurance(cursor, connection, df_us)

## Load cases covid

In [59]:
load_corona_cases(cursor, connection, df_cd)